In [3]:
import pandas_profiling
import pandas as pd

1. Проверка наличия/обработка пропусков
2. Проверьте взаимосвязи между признаками
3. Попробуйте создать свои признаки
4. Удалите лишние
5. Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда
6. Использование профайлера вам поможет.
7. Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)


In [37]:
data = pd.read_csv('aac_shelter_outcomes.csv')
data.shape

(78256, 12)

In [3]:
data.profile_report()

C:\ProgramData\Anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:124: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 2).')
  correlation_name=correlation_name, error=error


In [20]:
data.isnull().sum()

age_upon_outcome        8
animal_id               0
animal_type             0
breed                   0
color                   0
date_of_birth           0
datetime                0
monthyear               0
name                23886
outcome_subtype     42293
outcome_type           12
sex_upon_outcome        2
dtype: int64

некоторые признаки имеют пропуски. те которых малое количество 'age_upon_outcome', 'outcome_type', 'sex_upon_outcome' удаляю из данных, т.к. удаленный объем существенно не влияет на данные

In [38]:
data = data.dropna(axis=0, subset=['age_upon_outcome', 'outcome_type', 'sex_upon_outcome'])

есть еще два признакак с большим количеством пропущенных значений 'name', 'outcome_subtype'. в этом случае пропускам присваиваю значение 'unknow', так как необходимо сохранить эти данные в датасете и при этом обработать пропуски

In [39]:
data = data.fillna(value='unknow')

In [40]:
data.isnull().sum()

age_upon_outcome    0
animal_id           0
animal_type         0
breed               0
color               0
date_of_birth       0
datetime            0
monthyear           0
name                0
outcome_subtype     0
outcome_type        0
sex_upon_outcome    0
dtype: int64

данные имеют 10 дубликатов, которые удаляю

In [43]:
data = data.drop_duplicates()

проанализировав суть признаков, считаю что ID и имя животного не оказывают влияния на предсказываемое значение. Поэтому не использую их

In [46]:
data = data[['age_upon_outcome', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'monthyear', 'outcome_subtype',
       'outcome_type', 'sex_upon_outcome']]

в данных есть дублирующие столбцы 'datetime' и 'monthyear', также столбец 'age_upon_outcome' это разница столбцов 'datetime' и  'date_of_birth'

так как эти столбцы показывают по сути одно значения, я их удаляю. при этом столбец 'age_upon_outcome' представлю в виде числового значения, в днях. как разность  'datetime' и 'date_of_birth'

In [50]:
from datetime import datetime, date, time

сначала преобразовал признаки в которых содержится дата в тип данных datetime

In [61]:
data['date_of_birth'] =  pd.to_datetime(data['date_of_birth'])

In [65]:
data['datetime'] =  pd.to_datetime(data['datetime'])

In [67]:
data['monthyear'] =  pd.to_datetime(data['monthyear'])

после получения необходимого типа данных, нашел разнасть дат, таким образом решил две вещи:
1. получил более точный аналог признака age_upon_outcome
2. получил значения признака в числовом формате

In [91]:
data['days'] = data.datetime - data.date_of_birth

новый признак выразил в количестве дней

In [93]:
data['days'] = data['days'].dt.total_seconds() / 86400

выбираю итоговые признаки, которые буду использовать

In [96]:
data_mod = data[['animal_type', 'breed', 'color', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'days']]

In [97]:
data_mod.head()

,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,days
0,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male,15.669444
1,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female,366.490972
2,Dog,Pit Bull,Blue/White,unknow,Adoption,Neutered Male,429.597222
3,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male,3300.659722
4,Other,Bat Mix,Brown,Rabies Risk,Euthanasia,Unknown,181.586111


далее необходимо преобразовать категориальные признаки в числовые значения. Для этого применяю LableEncoder

In [98]:
from sklearn import preprocessing

In [99]:
list_ = ['animal_type', 'breed', 'color', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome']
column = preprocessing.LabelEncoder()
for i in list_:
    data_mod[i] = column.fit_transform(data_mod[i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [100]:
data_mod.head()

,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,days
0,1,937,321,12,8,1,15.669444
1,2,255,474,12,8,3,366.490972
2,2,1612,119,19,0,2,429.597222
3,2,1467,463,12,8,2,3300.659722
4,4,253,122,14,3,4,181.586111


разделяю данные для тренировки. Для этого отделяю предсказываемый признак 'outcome_type'

In [115]:
X = data_mod[['animal_type', 'breed', 'color', 'outcome_subtype', 'sex_upon_outcome', 'days']]
y = data_mod['outcome_type']

разделяю данные на тренировочные и тестовые

In [116]:
from sklearn.model_selection import train_test_split

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

создаю модель и оцениваю качество

In [109]:
from sklearn.ensemble import RandomForestClassifier

In [120]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [121]:
score = model.score(X_test, y_test)

In [122]:
score

0.8861050747794964

In [129]:
print(f'Качество модели {model.score(X_test, y_test)*100} %')

Качество модели 88.61050747794964 %


In [130]:
from sklearn.metrics import classification_report

In [131]:
model_pred = model.predict(X_test)

print(classification_report(y_test, model_pred))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87      9999
           1       0.90      0.82      0.86       209
           2       0.62      0.68      0.65        84
           3       0.98      0.97      0.98      1799
           4       0.40      0.22      0.29         9
           5       0.00      0.00      0.00         8
           6       0.72      0.67      0.70      4313
           7       0.00      0.00      0.00        42
           8       1.00      1.00      1.00      7006

    accuracy                           0.89     23469
   macro avg       0.61      0.58      0.59     23469
weighted avg       0.88      0.89      0.88     23469



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


результат модели получился 89%. Имеет высокие значения точности и полноты предсказания по самым встречающимся классам. За исключением класса 6

в другом подходе исключаю признак который сам создал и который надо предсказать, к остальным применяю onehotencoder

In [143]:
data_ohe = data[['animal_type', 'breed', 'color', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'days']]

In [145]:
data_ohe.head()

,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,days
0,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male,15.669444
1,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female,366.490972
2,Dog,Pit Bull,Blue/White,unknow,Adoption,Neutered Male,429.597222
3,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male,3300.659722
4,Other,Bat Mix,Brown,Rabies Risk,Euthanasia,Unknown,181.586111


In [146]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
onehotencoder = OneHotEncoder(sparse=False)
df = pd.DataFrame(onehotencoder.fit_transform(data_ohe[['animal_type', 'sex_upon_outcome', 'breed', 'outcome_subtype', 'days']]))

получив огромную размерность данных, с помощью pca сокращаю данные до 15 признаков

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=15)

In [ ]:
pca.fit(df)

In [ ]:
df_pca = pca.transform(df)

In [ ]:
df_pca = pd.DataFrame(df_pca)

к полученным 15 признакам добавляю, которые не менял 'days', 'outcome_type'

In [ ]:
df_pca['days'] = data_mod['days']

In [ ]:
df_pca['outcome_type'] = data_mod['outcome_type']

удаляю пропуски в добавленных признаках

In [ ]:
df_pca = df_pca.dropna(axis=0, subset=['days', 'outcome_type'])

перевожу столбец для предсказания в числовые категории с помощью LableEncoder

In [148]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()
df_pca['outcome_type'] = le.fit_transform(df_pca['outcome_type'])

разделяю данные на значения  и то что требуется предсказать

In [ ]:
X = df_pca.iloc[:, :15]
y = df_pca.iloc[:, -1]

разделяю данные на тренировочные и тестовые

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X, y, test_size=0.3)

создаю новую модель на другом подходе к данным, после onehotencoder и pca

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train_p, y_train_p)
score = model.score(X_test_p, y_test_p)

качество такой модели получилось 37%. думаю повлияло:
* разная размерность признаков
* смешивание различных типов данных
* большой разброс расмеров классов